## 직렬화(Serialization)

`is_lc_serializable` 클래스 메서드로 실행하여 LangChain 클래스가 직렬화 가능한지 확인할 수 있습니다.

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH04-Models")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH04-Models


In [4]:
# Bedrock 설정 및 Util 함수 설정
from langchain_aws import ChatBedrockConverse
import os
from Util.stream_utils import print_stream_content, get_stream_content

# 객체 생성
llm = ChatBedrockConverse(
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    model="apac.anthropic.claude-sonnet-4-20250514-v1:0",
    region_name="ap-northeast-2",
)

In [5]:
import os
from langchain.prompts import PromptTemplate

# 프롬프트 템플릿을 사용하여 질문을 생성합니다.
prompt = PromptTemplate.from_template("{fruit}의 색상이 무엇입니까?")

클래스(class) 에 대하여 직렬화 가능 여부를 확인합니다.

In [6]:
# 직렬화가 가능한지 체크합니다.
print(f"ChatBedrockConverse: {ChatBedrockConverse.is_lc_serializable()}")

ChatBedrockConverse: True


llm 객체에 대하여 직렬화 가능 여부를 확인합니다.

In [7]:
# 직렬화가 가능한지 체크합니다.
print(f"ChatBedrockConverse: {llm.is_lc_serializable()}")

ChatBedrockConverse: True


In [8]:
# 체인을 생성합니다.
chain = prompt | llm

# 직렬화가 가능한지 체크합니다.
chain.is_lc_serializable()

True

## 체인(Chain) 직렬화(dumps, dumpd)

### 개요

체인 직렬화는 직렬화 가능한 모든 객체를 딕셔너리 또는 JSON 문자열로 변환하는 과정을 의미합니다.

### 직렬화 방법

객체의 속성 및 데이터를 키-값 쌍으로 저장하여 딕셔너리 형태로 변환합니다.

이러한 직렬화 방식은 객체를 쉽게 저장하고 전송할 수 있게 하며, 다양한 환경에서 객체를 재구성할 수 있도록 합니다.

**참고**
- `dumps`: 객체를 JSON 문자열로 직렬화
- `dumpd`: 객체를 딕셔너리로 직렬화


In [9]:
from langchain_core.load import dumpd, dumps

dumpd_chain = dumpd(chain)
dumpd_chain

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'schema', 'runnable', 'RunnableSequence'],
 'kwargs': {'first': {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'prompts', 'prompt', 'PromptTemplate'],
   'kwargs': {'input_variables': ['fruit'],
    'template': '{fruit}의 색상이 무엇입니까?',
    'template_format': 'f-string'},
   'name': 'PromptTemplate'},
  'last': {'lc': 1,
   'type': 'constructor',
   'id': ['langchain_aws', 'chat_models', 'ChatBedrockConverse'],
   'kwargs': {'model_id': 'apac.anthropic.claude-sonnet-4-20250514-v1:0',
    'region_name': 'ap-northeast-2',
    'aws_access_key_id': {'lc': 1,
     'type': 'secret',
     'id': ['AWS_ACCESS_KEY_ID']},
    'aws_secret_access_key': {'lc': 1,
     'type': 'secret',
     'id': ['AWS_SECRET_ACCESS_KEY']},
    'provider': 'anthropic',
    'supports_tool_choice_values': ['auto', 'any', 'tool']},
   'name': 'ChatBedrockConverse'}},
 'name': 'RunnableSequence'}

In [10]:
# 직렬화된 체인의 타입을 확인합니다.
type(dumpd_chain)

dict

이번에는 `dumps` 함수를 사용하여 직렬화된 체인을 확인해보겠습니다.

In [11]:
# dumps 함수를 사용하여 직렬화된 체인을 확인합니다.
dumps_chain = dumps(chain)
dumps_chain

'{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "runnable", "RunnableSequence"], "kwargs": {"first": {"lc": 1, "type": "constructor", "id": ["langchain", "prompts", "prompt", "PromptTemplate"], "kwargs": {"input_variables": ["fruit"], "template": "{fruit}\\uc758 \\uc0c9\\uc0c1\\uc774 \\ubb34\\uc5c7\\uc785\\ub2c8\\uae4c?", "template_format": "f-string"}, "name": "PromptTemplate"}, "last": {"lc": 1, "type": "constructor", "id": ["langchain_aws", "chat_models", "ChatBedrockConverse"], "kwargs": {"model_id": "apac.anthropic.claude-sonnet-4-20250514-v1:0", "region_name": "ap-northeast-2", "aws_access_key_id": {"lc": 1, "type": "secret", "id": ["AWS_ACCESS_KEY_ID"]}, "aws_secret_access_key": {"lc": 1, "type": "secret", "id": ["AWS_SECRET_ACCESS_KEY"]}, "provider": "anthropic", "supports_tool_choice_values": ["auto", "any", "tool"]}, "name": "ChatBedrockConverse"}}, "name": "RunnableSequence"}'

In [12]:
# 직렬화된 체인의 타입을 확인합니다.
type(dumps_chain)

str

## Pickle 파일

### 개요

Pickle 파일은 Python 객체를 바이너리 형태로 직렬화하는 포맷입니다.

### 특징

1. **형식:**
   - Python 객체를 바이너리 형태로 직렬화하는 포맷

2. **특징:**
   - Python 전용 (다른 언어와 호환 불가)
   - 대부분의 Python 데이터 타입 지원 (리스트, 딕셔너리, 클래스 등)
   - 객체의 상태와 구조를 그대로 보존

3. **장점:**
   - 효율적인 저장 및 전송
   - 복잡한 객체 구조 유지
   - 빠른 직렬화/역직렬화 속도

4. **단점:**
   - 보안 위험 (신뢰할 수 없는 데이터 역직렬화 시 주의 필요)
   - 사람이 읽을 수 없는 바이너리 형식

### 주요 용도

1. 객체 캐싱
2. 머신러닝 모델 저장
3. 프로그램 상태 저장 및 복원

### 사용법

- `pickle.dump()`: 객체를 파일에 저장
- `pickle.load()`: 파일에서 객체 로드


pickle 파일로 저장합니다.

In [13]:
import pickle

# fuit_chain.pkl 파일로 직렬화된 체인을 저장합니다.
with open("fruit_chain.pkl", "wb") as f:
    pickle.dump(dumpd_chain, f)

JSON 형식으로 마찬가지로 저장할 수 있습니다.

In [14]:
import json

with open("fruit_chain.json", "w") as fp:
    json.dump(dumpd_chain, fp)

## load: 저장한 모델 불러오기


먼저, 이전에 저장한 `pickle` 형식의 파일을 로드합니다.

In [15]:
import pickle

# pickle 파일을 로드합니다.
with open("fruit_chain.pkl", "rb") as f:
    loaded_chain = pickle.load(f)

로드한 json 파일을 `load` 메서드를 사용하여 로드합니다.

In [16]:
from langchain_core.load import load

# 체인을 로드합니다.
chain_from_file = load(loaded_chain)

# 체인을 실행합니다.
print(chain_from_file.invoke({"fruit": "사과"}))

C:\Users\younghl\AppData\Local\Temp\ipykernel_8720\741098595.py:4: LangChainBetaWarning: The function `load` is in beta. It is actively being worked on, so the API may change.
  chain_from_file = load(loaded_chain)


content='사과의 색상은 품종에 따라 다양합니다:\n\n• **빨간색** - 가장 흔한 색상 (홍로, 후지 등)\n• **초록색** - 청사과라고 불리는 품종들\n• **노란색** - 골든 딜리셔스 등\n• **혼합색** - 빨강과 노랑이 섞인 것, 빨강과 초록이 섞인 것 등\n\n일반적으로 사람들이 사과 하면 떠올리는 색은 빨간색이지만, 실제로는 다양한 색상의 사과가 존재합니다.' additional_kwargs={} response_metadata={'ResponseMetadata': {'RequestId': '3ac86a2c-25b3-42f5-8f92-39badb4b3000', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 30 Jul 2025 09:32:23 GMT', 'content-type': 'application/json', 'content-length': '748', 'connection': 'keep-alive', 'x-amzn-requestid': '3ac86a2c-25b3-42f5-8f92-39badb4b3000'}, 'RetryAttempts': 0}, 'stopReason': 'end_turn', 'metrics': {'latencyMs': [3823]}, 'model_name': 'apac.anthropic.claude-sonnet-4-20250514-v1:0'} id='run--78f8666a-ef73-4b8c-9421-5bea56881af0-0' usage_metadata={'input_tokens': 23, 'output_tokens': 200, 'total_tokens': 223, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}}


In [17]:
from langchain_core.load import load, loads

load_chain = load(
    loaded_chain, secrets_map={"AWS_ACCESS_KEY_ID": os.environ["AWS_ACCESS_KEY_ID"], "AWS_SECRET_ACCESS_KEY": os.environ["AWS_SECRET_ACCESS_KEY"]}
)

# 불러온 체인이 정상 동작하는지 확인합니다.
load_chain.invoke({"fruit": "사과"})

AIMessage(content='사과의 색상은 품종에 따라 다양합니다:\n\n**주요 색상:**\n- **빨간색** - 가장 일반적 (홍옥, 후지 등)\n- **초록색** - 청사과, 그래니 스미스 등\n- **노란색** - 골든 델리셔스 등\n- **혼합색** - 빨강과 노랑, 빨강과 초록이 섞인 것\n\n대부분의 사람들이 떠올리는 대표적인 사과 색상은 **빨간색**이지만, 실제로는 품종마다 색상이 매우 다양합니다.', additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': '32db00b6-a8de-4982-8781-d2534ebb78ba', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 30 Jul 2025 09:36:24 GMT', 'content-type': 'application/json', 'content-length': '747', 'connection': 'keep-alive', 'x-amzn-requestid': '32db00b6-a8de-4982-8781-d2534ebb78ba'}, 'RetryAttempts': 0}, 'stopReason': 'end_turn', 'metrics': {'latencyMs': [3701]}, 'model_name': 'apac.anthropic.claude-sonnet-4-20250514-v1:0'}, id='run--d80336e8-16ea-446b-b89e-a850da29291a-0', usage_metadata={'input_tokens': 23, 'output_tokens': 201, 'total_tokens': 224, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}})

In [18]:
with open("fruit_chain.json", "r") as fp:
    loaded_from_json_chain = json.load(fp)
    loads_chain = load(loaded_from_json_chain)

In [19]:
# 불러온 체인이 정상 동작하는지 확인합니다.
loads_chain.invoke({"fruit": "사과"})

AIMessage(content='사과의 색상은 품종에 따라 다양합니다:\n\n**주요 색상들:**\n- **빨간색** - 가장 흔한 색상 (홍로, 후지 등)\n- **초록색** - 청사과, 그래니 스미스 등\n- **노란색** - 골든 딜리셔스 등\n- **혼합색** - 빨강과 노랑이 섞인 것, 빨강과 초록이 섞인 것 등\n\n대부분의 사람들이 떠올리는 전형적인 사과 색상은 **빨간색**이지만, 실제로는 품종마다 색상이 매우 다양합니다.', additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': 'd15a9e08-9ffa-4e55-a7e0-7bc0355f4cef', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 30 Jul 2025 09:36:47 GMT', 'content-type': 'application/json', 'content-length': '772', 'connection': 'keep-alive', 'x-amzn-requestid': 'd15a9e08-9ffa-4e55-a7e0-7bc0355f4cef'}, 'RetryAttempts': 0}, 'stopReason': 'end_turn', 'metrics': {'latencyMs': [3951]}, 'model_name': 'apac.anthropic.claude-sonnet-4-20250514-v1:0'}, id='run--157bf0fa-4bf6-4848-957b-9b1a517b0ea2-0', usage_metadata={'input_tokens': 23, 'output_tokens': 214, 'total_tokens': 237, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}})